In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

# Date preparation

Check if the missing values are presented in the features.
- If there are missing values:
- For categorical features, replace them with 'NA'
- For numerical features, replace with with 0.0

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.columns[df.isnull().any()]

Index(['lead_source', 'industry', 'annual_income', 'employment_status',
       'location'],
      dtype='object')

In [5]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [6]:
for col in df.columns:
    if df[col].isnull().any():
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('NA')
        else:
            df[col] = df[col].fillna(0.0)

In [7]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


# Question 1

What is the most frequent observation (mode) for the column industry?

- NA
- technology
- healthcare
- retail

In [8]:
df['industry'].mode()[0]

'retail'

# Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

number_of_courses_viewed, annual_income, interaction_count, lead_score

What are the two features that have the biggest correlation?

- interaction_count and lead_score
- number_of_courses_viewed and lead_score
- number_of_courses_viewed and interaction_count
- annual_income and interaction_count

Only consider the pairs above when answering this question.

In [10]:
cols = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

corr_matrix = df[cols].corr()
print(corr_matrix)

                          number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000  


The answer is annual_income and interaction_count (d)

# Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value converted is not in your dataframe.

In [12]:
# Assume your target column is called 'target'
X = df.drop(columns=['converted'])
y = df['converted']

# First split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Then split temp into val (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_val['annual_income']

# Question 3

- Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

- industry
- location
- lead_source
- employment_status

In [17]:
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

for col in categorical_cols:
    score = mutual_info_score(X_train[col], y_train)
    print(f"{col}: {round(score, 2)}")

lead_source: 0.03
industry: 0.02
employment_status: 0.02
location: 0.0


answer: lead_source (c)

# Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
  - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  - model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

0.64
0.74
0.84
0.94

In [20]:
# One-hot encode categorical variables
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
#encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train[categorical_cols])
X_val_cat = encoder.transform(X_val[categorical_cols])

# Drop categorical columns from original data and concatenate one-hot encoded columns
X_train_num = X_train.drop(columns=categorical_cols).values
X_val_num = X_val.drop(columns=categorical_cols).values

X_train_final = np.hstack([X_train_num, X_train_cat])
X_val_final = np.hstack([X_val_num, X_val_cat])

# Train logistic regression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_final, y_train)

# Predict and calculate accuracy
y_pred = model.predict(X_val_final)
acc = accuracy_score(y_val, y_pred)
print(round(acc, 2))

0.74


# Question 5

- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

'industry'
'employment_status'
'lead_score'
Note: The difference doesn't have to be positive.

In [21]:
# List of features to test
features = X_train.columns.tolist()

# One-hot encode categorical variables
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train[categorical_cols])
X_val_cat = encoder.transform(X_val[categorical_cols])
X_train_num = X_train.drop(columns=categorical_cols).values
X_val_num = X_val.drop(columns=categorical_cols).values
X_train_full = np.hstack([X_train_num, X_train_cat])
X_val_full = np.hstack([X_val_num, X_val_cat])

# Train model with all features
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_full, y_train)
y_pred = model.predict(X_val_full)
original_acc = accuracy_score(y_val, y_pred)

# Store accuracy differences
acc_diffs = {}

for feature in features:
    # Exclude feature
    X_train_drop = X_train.drop(columns=[feature])
    X_val_drop = X_val.drop(columns=[feature])
    # One-hot encode
    cat_cols_drop = [col for col in X_train_drop.columns if X_train_drop[col].dtype == 'object']
    encoder_drop = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_train_cat_drop = encoder_drop.fit_transform(X_train_drop[cat_cols_drop])
    X_val_cat_drop = encoder_drop.transform(X_val_drop[cat_cols_drop])
    X_train_num_drop = X_train_drop.drop(columns=cat_cols_drop).values
    X_val_num_drop = X_val_drop.drop(columns=cat_cols_drop).values
    X_train_final = np.hstack([X_train_num_drop, X_train_cat_drop])
    X_val_final = np.hstack([X_val_num_drop, X_val_cat_drop])
    # Train and evaluate
    model_drop = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_drop.fit(X_train_final, y_train)
    y_pred_drop = model_drop.predict(X_val_final)
    acc_drop = accuracy_score(y_val, y_pred_drop)
    acc_diffs[feature] = round(original_acc - acc_drop, 4)

# Print accuracy differences
for feature, diff in acc_diffs.items():
    print(f"{feature}: {diff}")

lead_source: 0.0137
industry: 0.0
number_of_courses_viewed: 0.0651
annual_income: -0.113
employment_status: -0.0034
location: 0.0
interaction_count: 0.0685
lead_score: 0.0


Multiple answers... Go with industry...

# Question 6

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10
100


In [32]:
# One-hot encode categorical variables
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train[categorical_cols])
X_val_cat = encoder.transform(X_val[categorical_cols])
X_train_num = X_train.drop(columns=categorical_cols).values
X_val_num = X_val.drop(columns=categorical_cols).values
X_train_final = np.hstack([X_train_num, X_train_cat])
X_val_final = np.hstack([X_val_num, X_val_cat])

for C in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_final, y_train)
    y_pred = model.predict(X_val_final)
    acc = accuracy_score(y_val, y_pred)
    print(f"C={C}: {round(acc, 5)}")

C=0.01: 0.74315
C=0.1: 0.74315
C=1: 0.74315
C=10: 0.74315
C=100: 0.74315


answer: ?

In [34]:
X_train_final[:5]

array([[1.0000e+00, 6.1705e+04, 4.0000e+00, 6.5000e-01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.0000e+00, 5.5199e+04, 4.0000e+00, 9.0000e-02, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.0000e+00, 4.0841e+04, 4.0000e+00, 6.1000e-01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
      